<a href="https://colab.research.google.com/github//pylabel-project/samples/blob/main/pylabeler.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> 

## PyLabeler: Label images with PyLabel, YOLOv5, and jupyter-bbox-widget. 

This notebook is a labeling tool you can use to 
annotate image datasets with bounding boxes, build an object detection model to automatically suggest bounding boxes, and save the annotations in YOCO, COCO, or VOC format. The annotation interface uses the [jupyter-bbox-widget](https://github.com/gereleth/jupyter-bbox-widget). The bounding box detection uses PyTorch and a [VOLOv5](https://github.com/ultralytics/yolov5) model.

Sample iterative labeling workflow:

1. Manually label several images and save the annotations. 
2. Train an object detection model using the annotations. 
3. Use the newly trained model to automatically suggest labels for additional images. Adjust labels as needed. 
Repeat steps 2 and 3 until model the model-predicted values achieve the desired level of accuracy. 

In [12]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)
!pip install pylabel > /dev/null. 
from pylabel import importer
import os 
import zipfile

from pylabel import importer

## Import Images to be Labeled 
For this demonstration we will download 100 images from the <a href="https://github.com/pylabel-project/datasets_models#squirrels-and-nuts">squirrels and nuts dataset.</a>


In [13]:
os.makedirs("data/", exist_ok=True)
!wget "https://github.com/pylabel-project/datasets_models/blob/main/squirrelsandnuts/squirrelsandnuts_train.zip?raw=true" -O data/squirrelsandnuts_train.zip
with zipfile.ZipFile("data/squirrelsandnuts_train.zip", 'r') as zip_ref:
    zip_ref.extractall("data/")
dataset = importer.ImportImagesOnly("data/squirrelsandnuts_train/images/train", ends_with=".jpeg")

--2021-11-28 06:25:11--  https://github.com/pylabel-project/datasets_models/blob/main/squirrelsandnuts/squirrelsandnuts_train.zip?raw=true
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/pylabel-project/datasets_models/raw/main/squirrelsandnuts/squirrelsandnuts_train.zip [following]
--2021-11-28 06:25:12--  https://github.com/pylabel-project/datasets_models/raw/main/squirrelsandnuts/squirrelsandnuts_train.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pylabel-project/datasets_models/main/squirrelsandnuts/squirrelsandnuts_train.zip [following]
--2021-11-28 06:25:12--  https://raw.githubusercontent.com/pylabel-project/datasets_models/main/squirrelsandnuts/squirrelsandnuts_train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com

# Load YOLO Model
Install the dependencies required by YOLOv5, which in used for bounding box prediction and training new models. 

In [15]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install
%cd ../
import torch
from yolov5 import utils

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5
/content


Load a YOL0v5 object detection model that you can use automatically label images. Use the below widgets to select and configure the models:
- **model_path:** 
  - Select yolov5s to load the default YOLO model trained on the COCO dataset. It is a good general purpose model but it doesn't have classes for squirrels and nuts. Give it a try and see what happens. 
  - Select *custom.pt* to load the weights from another pre-trained model. In this example you can load a model that was trained on images of squirrels and nuts. 
  - Select *yolov5/runs/train/exp/weights/best.pt* to load a new model that you have trained (using the steps further below in the notebook). 

If you load a model that is not many any predictions, try lowering the model_conf value. 

In [16]:
#Download custom weights that were pre-trained for this task
!wget https://github.com/pylabel-project/datasets_models/blob/main/squirrelsandnuts/squirrelsandnuts_yolov5.pt?raw=true -O custom.pt

model_path = "custom.pt" #@param ["yolov5s", "yolov5/runs/train/exp/weights/best.pt", "custom.pt"] {allow-input: true}
model_conf = 0.25 #@param {type:"slider", min:0, max:1, step:0.01}
model_iou = .45 #@param {type:"slider", min:0, max:1, step:0.01}

model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True) 

model.conf = model_conf
model.iou = model_iou

--2021-11-28 06:25:55--  https://github.com/pylabel-project/datasets_models/blob/main/squirrelsandnuts/squirrelsandnuts_yolov5.pt?raw=true
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/pylabel-project/datasets_models/raw/main/squirrelsandnuts/squirrelsandnuts_yolov5.pt [following]
--2021-11-28 06:25:55--  https://github.com/pylabel-project/datasets_models/raw/main/squirrelsandnuts/squirrelsandnuts_yolov5.pt
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pylabel-project/datasets_models/main/squirrelsandnuts/squirrelsandnuts_yolov5.pt [following]
--2021-11-28 06:25:55--  https://raw.githubusercontent.com/pylabel-project/datasets_models/main/squirrelsandnuts/squirrelsandnuts_yolov5.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip


requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...


YOLOv5 🚀 2021-11-28 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)




requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


## Predict and Edit Annotations
Use the jupyter_bbox_widget to inspect, edit, and save annotations without leaving the Jupyter notebook. Press predict to autolabel images using a pretrained model. For instructions and keyboard shortcuts for using this widget see https://github.com/gereleth/jupyter-bbox-widget#Usage.

In [17]:
dataset.labeler.StartPyLaber(yolo_model=model)

Run the below cell to check how many example of each class you have annotated. It is recomended to label at least 10 examples of each class before using the annotations to train a new model. 

In [18]:
dataset.analyze.class_counts

            92
Nut         19
Squirrel     5
Name: cat_name, dtype: int64

Export the annotations so you can use them to train a new model. 

In [19]:
dataset.export.ExportToYoloV5(output_path='training/labels', yaml_file='dataset.yaml',copy_images=True)

['training/dataset.yaml',
 'training/labels/2021-07-03T06-47-39-frame_0000.txt',
 'training/labels/2021-07-03T07-52-45-frame_0002.txt',
 'training/labels/2021-07-03T07-55-20-frame_0003.txt',
 'training/labels/2021-07-03T07-55-47-frame_0006.txt',
 'training/labels/2021-07-03T06-47-39-frame_0001.txt',
 'training/labels/2021-07-03T07-55-47-frame_0001.txt',
 'training/labels/2021-07-03T06-41-41-frame_0000.txt',
 'training/labels/2021-07-03T07-13-34-frame_0003.txt']

Use the annotations you just created to train a new model. It is recomended to train new models for at least 100 epochs. 

In [20]:
%cd yolov5
!python train.py --img 960 --batch 20 --epochs 10 --data ../training/dataset.yaml --weights yolov5s.pt --cache --exist-ok
%cd ../

/content/yolov5
train: weights=yolov5s.pt, cfg=, data=../training/dataset.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=20, imgsz=960, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=True, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-110-gfcd180d torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degre

After the model has completed training: 
- Review the model summary above. Check the precision, recall, and mAP values for all of the classes. If there is a class that has worse values than the others, annotate more samples for that class. 
- The model weights will be saved to *runs/train/exp/weights/best.pt*. To perform interative labeling and training reload this model into the labeling tool and use it to train additional images. Repeat and label additional images until desired results are achieved. 